In [7]:
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms
import numpy as np
import pandas as pd

In [8]:
!pip install -q SimpleITK
import SimpleITK as sitk

In [1]:
!pip install -q torchsummary
from torchsummary import summary

In [22]:
class ThreeDimCNN(nn.Module):
    def __init__(self):
        super(ThreeDimCNN, self).__init__()

        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=1)
        # self.conv2 = nn.Conv3d(8, 16, kernel_size=3, stride=1, padding=1)
        # self.conv3 = nn.Conv3d(16, 32, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool3d(kernel_size=8, stride=8)

        self.dropout = nn.Dropout(p=0.2)

        self.fc = nn.Linear(524288, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.flatten = nn.Flatten()

    def forward(self, x):
        print(x.shape)
        x = self.relu(self.conv1(x))
        x = self.pool(x)

#         x = self.relu(self.conv2(x))
#         x = self.pool(x)

#         x = self.relu(self.conv3(x))
#         x = self.pool(x)
        print(x.shape)

        x = self.flatten(x)
        print(x.shape)

        x = self.fc(x)
        x = self.dropout(x)
        x = self.sigmoid(x)

        return x

In [23]:
device = torch.device("cpu")
ct_scan = sitk.ReadImage("./1.3.6.1.4.1.14519.5.2.1.6279.6001.100530488926682752765845212286.mhd")
data = sitk.GetArrayFromImage(ct_scan)
data = np.expand_dims(data, 0)
data = np.concatenate((data, data))
# print(data.shape)
data = torch.from_numpy(data).to(device)
data = torch.unsqueeze(data, 1)
print(data.shape)

torch.Size([2, 1, 256, 256, 256])


In [24]:
loss_fn = nn.BCELoss()
with torch.no_grad():
    model = ThreeDimCNN().to(device)
    label = torch.tensor([[1], [1]]).float().to(device)
    output = model(data)
    loss = loss_fn(output, label)
    print(loss)


torch.Size([2, 1, 256, 256, 256])
torch.Size([2, 16, 32, 32, 32])
torch.Size([2, 524288])
tensor(0.6274)


In [16]:
model = ThreeDimCNN().to("cpu")

print(model)

ThreeDimCNN(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=4194304, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [ ]:
m = nn.Sigmoid()
input = torch.randn(1)
output = m(input)
output

In [ ]:
metadata = pd.read_csv("./metadata.csv")

In [ ]:
sample_metadata = metadata[metadata["class"] == 0].sample(n=1)

In [ ]:
file_name = sample_metadata['seriesuid'].values[0]
label = sample_metadata['class'].values[0]

In [ ]:
label